# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [16]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [17]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    R1 = (data['Price in 1 Year']/P)-1
    prob = data['Probability']
    ex_r = (prob * R1).sum()
    return ex_r
#print(exp_return(data, 40))

0.07500000000000002


Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [18]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    ex_r = (data['Price in 1 Year'] * data['Probability']).sum()
    var = ((data["Price in 1 Year"] - ex_r) ** 2 * data["Probability"]).sum()
    sd = (var **(1/2))/P
    return sd
#print(volatility(data, 40))

0.21301408404140793


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [19]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    SD_error = volatility(data, P) / (len(data)**0.5)
    lower_b = exp_return(data, P) - (1.96 * SD_error)
    upper_b = exp_return(data, P) + (1.96 * SD_error)
    return [lower_b,upper_b]
#print(interval(data, 40))

[-0.16604812797447732, 0.31604812797447734]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [20]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    ex_r = rf + beta*(market_return - rf)
    return ex_r
#print(exp_return_capm(-0.6,0.1, 0.02))

-0.028


### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [21]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)
    df['RET'] = df['RET'].apply(pd.to_numeric,errors = 'coerce')
    df = df[pd.notnull(df['RET'])]
    count = df['PERMNO'].value_counts()
    valid = count.index[count >= 500]
    df = df[df['PERMNO'].isin(valid)]
    return df

#clean_data(r"/Users/hanyinwang/Downloads/monthlycrsp_mod6.csv")

PERMNO        date     CUSIP       RET
9027      10065  1970-01-30  00621210 -0.144375
9028      10065  1970-02-27  00621210  0.105882
9029      10065  1970-03-31  00621210  0.017857
9030      10065  1970-04-30  00621210 -0.078947
9031      10065  1970-05-29  00621210 -0.020952
...         ...         ...       ...       ...
3393558   84129  2022-08-31  98970110  0.016315
3393559   84129  2022-09-30  98970110 -0.075777
3393560   84129  2022-10-31  98970110  0.021235
3393561   84129  2022-11-30  98970110  0.005583
3393562   84129  2022-12-30  98970110 -0.051332

[403316 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [22]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    avg_r = data.groupby('PERMNO')['RET'].mean().reset_index()
    avg_r.columns = ['PERMNO','Avg_Return']
    output = pd.DataFrame(data = {
        "CUSIP" : avg_r['PERMNO'],
        "Average Return" : avg_r['Avg_Return']
    })
    return output
data = clean_data(r'/Users/hanyinwang/Downloads/monthlycrsp_mod6.csv')
#print(avg_return(data))

     CUSIP  Average Return
0    10065        0.009364
1    10145        0.012476
2    10225        0.013279
3    10516        0.014130
4    10656        0.009859
..     ...             ...
674  81614        0.015500
675  81702        0.015024
676  82107        0.011280
677  83601        0.016190
678  84129        0.012356

[679 rows x 2 columns]


In [23]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    volat = data.groupby('PERMNO')['RET'].std().reset_index()
    volat.columns = ['PERMNO','Volatility']
    output = pd.DataFrame(data = {
        "CUSIP" : volat['PERMNO'],
        "Volatility" : volat['Volatility']
    })
    return output
data = clean_data(r'/Users/hanyinwang/Downloads/monthlycrsp_mod6.csv')
#print(security_volatilities(data))

     CUSIP  Volatility
0    10065    0.049088
1    10145    0.080908
2    10225    0.072392
3    10516    0.084347
4    10656    0.117598
..     ...         ...
674  81614    0.167238
675  81702    0.111982
676  82107    0.073001
677  83601    0.103132
678  84129    0.095243

[679 rows x 2 columns]


Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [24]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    mer_data = pd.merge(ret_data, vol_data, on = 'CUSIP')
    correlat = mer_data['Average Return'].corr(mer_data['Volatility'])
    return correlat
data = clean_data(r'/Users/hanyinwang/Downloads/monthlycrsp_mod6.csv')
ret_data = avg_return(data)
vol_data = security_volatilities(data)
#print(correlation(ret_data, vol_data))

0.44837090727059614
